In [ ]:
!nvidia-smi

Mon Oct  3 14:22:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install comet-ml
!pip install scikit-multilearn
!pip install transformers 
!pip install datasets
!pip install nlp


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 394 kB 5.0 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 162 kB 60.0 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 495 kB 61.3 MB/s 
     |████████████████████████████████| 140 kB 63.5 MB/s 
     |████████████████████████████████| 127 kB 63.4 MB/s 
     |████████████████████████████████| 158 kB 67.6 MB/s 
     |████████████████████████████████| 157 kB 64.9 MB/s 
     |████████████████████████████████| 157 kB 71.0 MB/s 
     |████████████████████████████████| 157 kB 72.7 MB/s 
     |████████████████████████████████| 157 kB 67.1 MB/s 
     |████████████████████████████████| 157 kB 55.0 MB/s 
     |████████████████████████████████| 157 kB 73.8 MB/s 
     |████████████████████████████████| 157 kB 73.7 MB/s 
     |██████████████████████████████

In [ ]:
!curl --header 'Host: s122vlx.storage.yandex.net' --user-agent 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:105.0) Gecko/20100101 Firefox/105.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --referer 'https://disk.yandex.ru/' --cookie '_yasc=GtN1cutWxugTxS3Z+/h0C0QanEU8UG8W+R/EoC4FSNA0eQ==' --header 'Upgrade-Insecure-Requests: 1' --header 'Sec-Fetch-Dest: iframe' --header 'Sec-Fetch-Mode: navigate' --header 'Sec-Fetch-Site: cross-site' --header 'Sec-GPC: 1' 'https://s122vlx.storage.yandex.net/rdisk/aff8130e11352c51fa219841a4c098da22fc987d490d7befd6dfb9abcefd0589/633b0252/fKqInKw3d7bLFOeFnMGnhIy3B_bphgo1pQCkJE8soimeWLDvY1dliq-k_HB7kFLvAQUtj9KZasSLVs7sTIippy1bnVzdXZfCiH_li6hDcwWr8npumZHI4midPdWhecNq?uid=1130000042241254&filename=%D0%92%D0%BE%D0%BB%D0%BE%D0%B3%D0%B4%D0%B0.zip&disposition=attachment&hash=&limit=0&content_type=application%2Fzip&owner_uid=1130000042241254&fsize=1513031&hid=5b4c07a8deb74eabc21c6e06803e2e89&media_type=compressed&tknv=v2&etag=5510f7b29e569ba6eea343123b796ebd&rtoken=fTyJcGPd3rwC&force_default=yes&ycrid=na-7e8976b129f4b00fbdf103b45719be1d-downloader12f&ts=5ea232827b880&s=4494746840f2293eb244a8210eaed8ff72adb5d9a28ec1b4c43d22d0fe60905a&pb=U2FsdGVkX19psHB3K6Z2JpcvmBFwQKyS1lO08ilEeDou-XE500idVDsCMLPr1qtP18qnmDGVYpC7pnf85dX_nsLKl6CYAVKlYBek8f6S4DnFS3O-C7KAs4LPXgts23g-' --output 'Вологда.zip'
!unzip Вологда.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1477k  100 1477k    0     0   804k      0  0:00:01  0:00:01 --:--:--  803k
Archive:  Вологда.zip
   creating: Вологда/
  inflating: Вологда/sample_solution.csv  
  inflating: Вологда/test_dataset_test.csv  
  inflating: Вологда/train_dataset_train.csv  
   creating: Вологда/.ipynb_checkpoints/
  inflating: Вологда/.ipynb_checkpoints/vologda_baseline-checkpoint.ipynb  


In [ ]:
import comet_ml
comet_ml.init(project_name='vologda')

import torch
from pathlib import Path
from datasets import load_dataset
from transformers import AutoTokenizer,AutoModel, AutoModelForSequenceClassification,AutoModelForTokenClassification,TrainingArguments, Trainer
from tqdm.auto import tqdm
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import log_loss
from sklearn.preprocessing import MultiLabelBinarizer

np.random.seed(1337)
torch.manual_seed(1337)

COMET INFO: Comet API key is valid


In [ ]:
from transformers import BertForMaskedLM,BertTokenizerFast, pipeline, BatchEncoding,BertForTokenClassification,BertForSequenceClassification
from tokenizers import Encoding
model_ckpt = "sberbank-ai/sbert_large_nlu_ru" 
#model_ckpt = "sberbank-ai/ruBert-large" 
#model_ckpt = "yandex/RuLeanALBERT"
#model_ckpt = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, problem_type="multi_label_classification")

Downloading:   0%|          | 0.00/323 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/655 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
df_train = pd.read_csv('Вологда/train_dataset_train.csv')

In [ ]:
df_train["len_reviews"] = df_train["Отзыв"].map(len)

In [ ]:
train = df_train.copy().drop(columns = 'RecordNo',inplace = False)
train.fillna(0)

,Название книги,Автор,Ссылка на литрес,Рейтинг,Количество оценок,Количество отзывов,Имя читателя,Оценка книги читателем (из 5 баллов),Отзыв,Лайки на отзыв,Дислайки на отзыв,Релевантность,Таксономия релевантные,Таксономия не релевантные,Длина отзыва,Ценности,len_reviews
0,Зулейха открывает глаза,Гузель Яхина,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.7,3922,408,Айгуль Ляпина,5.0,Рекомендую книгу в прочтению/прослушиванию. Ес...,0,3,0,0,0,0,0,157
1,Зулейха открывает глаза,Гузель Яхина,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.6,24719,2103,Olga T,5.0,"Удивительно, что сейчас возникает ТАКАЯ литера...",0,1,0,0,0,0,1,283
2,Дети мои,Гузель Яхина,https://www.litres.ru/guzel-yahina/deti-moi/,4.4,8032,702,Кирилл Чириков,5.0,"Душевно, жизненно, чувственно, проникновенно!!...",0,0,0,1,0,0,1,278
3,ПОСТ,Дмитрий Глуховский,https://www.litres.ru/dmitriy-gluhovskiy/post/,3.9,430,48,Kimetov Oleg,5.0,Очень понравилось. Причем понравилось больше «...,2,7,0,1,0,0,0,342
4,ПОСТ,Дмитрий Глуховский,https://www.litres.ru/dmitriy-gluhovskiy/post-...,4.4,832,113,Сергей Никитин,5.0,Круто. Автор и он же чтец – гений. Прослушал н...,1,0,0,0,0,0,0,177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4834,Дети мои,Гузель Яхина,https://www.litres.ru/guzel-yahina/deti-moi/,4.4,8032,702,417504557,4.0,Ещё один замечательный роман Гузель Яхиной! По...,0,0,0,0,0,0,0,149
4835,Авиатор,Евгений Водолазкин,https://www.litres.ru/evgeniy-vodolazkin/aviator/,4.5,3551,260,Viktoria Steinbach,0.0,Как всегда у Водолазкина – увлекательнейший сю...,0,1,1,1,0,0,1,468
4836,Текст,Дмитрий Глуховский,https://www.litres.ru/dmitriy-gluhovskiy/tekst...,4.5,1923,246,AnVer,5.0,"Мрачно, грустно. Давно такого не читала и хват...",1,2,0,0,1,0,0,148
4837,Зулейха открывает глаза,Гузель Яхина,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.6,24719,2103,Мария Перешивко,5.0,Очень интересная книга! Как будто очутилась в ...,0,1,0,0,0,0,0,139


In [ ]:
cat_features = ['Название книги','Автор','Имя читателя','Рейтинг']
text_features = ['Отзыв']
target_columns = ['Релевантность',
       'Таксономия релевантные', 'Таксономия не релевантные',
       'Длина отзыва', 'Ценности']

train_labels = train.copy()[target_columns].values
train_data  = train.copy().drop(columns = target_columns,inplace = False)
train_data[cat_features] = train_data[cat_features].astype(str)
train_data[text_features] = train_data[text_features].astype(str)

In [ ]:
def create_text_from_cols(df):
  texts = pd.DataFrame()
  texts['text'] = [("Автор: " + str(t['Автор']) + ' ' + "Имя читателя: " + str(t['Имя читателя']) + ' ' +  'Отзыв: ' + str(t['Отзыв']) + ' Рейтинг: ' + str(t['Рейтинг']) + ' Лайки: ' +str(t['Лайки на отзыв']) 
                    + ' Дислайки:' + str(t['Дислайки на отзыв']) ) for index,t in df.iterrows()]
  return texts

In [ ]:
from skmultilearn.model_selection import iterative_train_test_split

In [ ]:
texts = create_text_from_cols(train)
X_train, y_train, X_val, y_val = iterative_train_test_split((texts['text']).values.reshape(-1,1), np.array(train_labels), test_size=0.1)
test = pd.read_csv('Вологда/test_dataset_test.csv').fillna(0)
test_texts = create_text_from_cols(test)

In [ ]:
train_encodings = tokenizer([x[0] for x in X_train], truncation=True, padding=True,max_length=256)
val_encodings = tokenizer([x[0] for x in X_val], truncation=True, padding=True,max_length=256)
test_encodings = tokenizer(list(test_texts['text']), truncation=True, padding=True,max_length=256)


In [ ]:
import torch

class HHDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = np.array(labels,dtype = float)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).float()
        return item

    def __len__(self):
        return len(self.labels)




train_dataset = HHDataset(train_encodings, y_train)
val_dataset = HHDataset(val_encodings, y_val)
test_dataset = HHDataset(test_encodings, np.zeros((len(test_texts),5)))
#small_dataset = HHDataset(small_enc, y_val[:100,:])

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=5, problem_type="multi_label_classification").to('cuda')

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--sberbank-ai--sbert_large_nlu_ru/snapshots/28d04bde633f23feb22916430a01cdfcadfd35e9/config.json
Model config BertConfig {
  "_name_or_path": "sberbank-ai/sbert_large_nlu_ru",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_

In [ ]:
args = TrainingArguments(".", num_train_epochs=3,fp16=True,report_to='comet_ml',learning_rate = 3e-5,per_device_eval_batch_size=8,evaluation_strategy="steps",logging_strategy="steps",logging_steps = 400,save_steps = 2000, per_device_train_batch_size=8)
trainer = Trainer(model=model, args=args ,train_dataset=train_dataset, eval_dataset=val_dataset, tokenizer=tokenizer)


using `logging_steps` to initialize `eval_steps` to 400
PyTorch: setting up devices
Using cuda_amp half precision backend


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4355
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1635
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_G

Step,Training Loss,Validation Loss
400,0.342300,0.220406
800,0.159000,0.181783


***** Running Evaluation *****
  Num examples = 484
  Batch size = 8
***** Running Evaluation *****
  Num examples = 484
  Batch size = 8


Step,Training Loss,Validation Loss
400,0.342300,0.220406
800,0.159000,0.181783
1200,0.109800,0.138304
1600,0.071700,0.146650


***** Running Evaluation *****
  Num examples = 484
  Batch size = 8
***** Running Evaluation *****
  Num examples = 484
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/kglnsk/vologda/4e5eacabc435484da1860ee51b10e13f
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     epoch [9]                   : (0.73, 3.0)
COMET INFO:     eval_loss [4]               : (0.138303741812706, 0.22040627896785736)
COMET INFO:     eval_runtime [4]            : (7.6847, 7.7256)
COMET INFO:     eval_samples_per_second [4] : (62.649, 62.983)
COMET INFO:     eval_steps_per_second [4]   : (7.896, 7.938)
COMET INFO:     learning_rate [4]           : (

TrainOutput(global_step=1635, training_loss=0.16851554602293428, metrics={'train_runtime': 915.8792, 'train_samples_per_second': 14.265, 'train_steps_per_second': 1.785, 'total_flos': 6087903339548160.0, 'train_loss': 0.16851554602293428, 'epoch': 3.0})

In [ ]:
preds = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 2075
  Batch size = 8


In [ ]:
probs = torch.sigmoid(torch.Tensor(preds.predictions))
pd.DataFrame(probs.numpy()).to_csv('sbert-3-preds.csv')

In [ ]:
answers = 1*((probs.numpy()>0.4))

In [ ]:
answers.shape

(2075, 5)

In [ ]:
df_submission = pd.read_csv('Вологда/sample_solution.csv')

In [ ]:
df_submission.iloc[:,1:] = answers
df_submission.to_csv('answers-sbert-3.csv',index = False)